In [3]:
import inspect
import torch
import importlib
from torch import nn
from torch.nn import functional as F
import torch.optim.lr_scheduler as lrs

import pytorch_lightning as pl

# from transformers import LlamaForCausalLM, LlamaTokenizer
import random
from pandas.core.frame import DataFrame
import os.path as op
import os
import numpy as np
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, LoraConfig, TaskType, PeftModel
import json
from unsloth import FastLanguageModel 
import vllm

WARNING 08-01 23:15:39 _custom_ops.py:14] Failed to import from vllm._C with ImportError('/home/tiger/.local/lib/python3.9/site-packages/vllm/_C.abi3.so: undefined symbol: _ZN3c104impl3cow11cow_deleterEPv')


In [5]:
model, tokenizer = FastLanguageModel.from_pretrained("/mnt/bn/data-tns-live-llm/leon/datasets/llama-2-7b-bnb-4bit",dtype = torch.bfloat16,load_in_4bit = True)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA H100 80GB HBM3. Max memory: 79.109 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 9.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.22.post7. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
os.environ["TOKENIZERS_PARALLELISM"]="false"

In [9]:
import copy
tmp = copy.deepcopy(model)

KeyError: 'module'

In [ ]:
model.save_pretrained_merged("/mnt/bn/data-tns-live-llm/leon/datasets/rec/score_model", tokenizer, save_method = "merged_16bit",)

INFO 08-01 23:19:43 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='/mnt/bn/data-tns-live-llm/leon/datasets/rec/score_model', speculative_config=None, tokenizer='/mnt/bn/data-tns-live-llm/leon/datasets/rec/score_model', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=/mnt/bn/data-tns-live-llm/leon/datasets/rec/score_model, use_v2_block_manager=False, enable_prefix_caching=False)
INFO 08-01 23:19:43 selector.py:189] Ca

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 08-01 23:19:46 model_runner.py:692] Loading model weights took 12.5523 GB
ERROR 08-01 23:19:46 _custom_ops.py:42] Error in calling custom op rms_norm: '_OpNamespace' '_C' object has no attribute 'rms_norm'
ERROR 08-01 23:19:46 _custom_ops.py:42] Possibly you have built or installed an obsolete version of vllm.
ERROR 08-01 23:19:46 _custom_ops.py:42] Please try a clean build and install of vllm,or remove old built files such as vllm/*cpython*.so and build/ .


AttributeError: '_OpNamespace' '_C' object has no attribute 'rms_norm'